<a href="https://colab.research.google.com/github/willismax/MediaSystem-Python-Course/blob/main/04.Selenium/%E7%B6%B2%E9%A0%81%E6%93%B7%E5%8F%96_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網頁擷取_Selenium

![image.png](https://hackmd.io/_uploads/H1hC-R1ma.png)



- 官方網站介紹
    - [Selenium](https://www.selenium.dev/)
    - [下載Driver](https://www.selenium.dev/selenium/docs/api/py/index.html) (4.X版已經不用額外安裝)


選擇元素的各種方法
```python
find_element(By.ID, "fruits")
find_element(By.NAME, "q")
find_element(By.CLASS_NAME, "tomatoes")
find_elements(By.TAG_NAME, "li")          #html標籤
find_element(By.LINK_TEXT, "More information...")
find_element(By.PARTIAL_LINK_TEXT, )
find_element(By.XPATH, )           #最常用    #/html開頭為絕對路徑；//相對路徑
find_element(By.CSS_SELECTOR,"#fruits .tomatoes")

```


In [ ]:
#下載安裝相關套件，以下為Colab的安裝方式

%pip install -U selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

### 常用的`webdriver.ChromeOptions()`參數
- start-maximized: Chrome開啟最大化視窗
- incognito: 無痕模式開啟
- headless: 不開啟Chrome模式
- disable-extensions: 停用Chrome擴充功能
- disable-popup-blocking: 停用Chrome彈出視窗
- make-default-browser: 設為預設瀏覽器
- version: 顯示瀏覽器版本
- disable-infobars: 阻止 Chrome 顯示通知“Chrome 由自動化軟體控制”

In [4]:
#@title 以 Selenium 爬取 PTT 指定目標

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


options = Options()
options.add_argument("--headless")
options.add_argument('--no-sandbox') #在背景執行
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--start-maximized")



wd = Chrome(options=options)
url = "https://www.ptt.cc/bbs/Gossiping/M.1750915957.A.39F.html"
wd.get(url)

while (wd.current_url=='https://www.ptt.cc/ask/over18?from=%2Fbbs%2FGossiping%2FM.1750915957.A.39F.html'):  #跳轉詢問18歲詢問頁面時，注意URL?from的變化
  wd.find_element(By.CLASS_NAME, "over18-button-container").click()

# 改XPATH定位
# 改SELECTOR定位


elem = wd.find_element(By.LINK_TEXT,'https://i.imgur.com/7cSIbP4.jpeg')
print(elem.text)
# wd.close()


https://i.imgur.com/7cSIbP4.jpeg




---



### 以 Selenium 搜尋 PTT 指定列表


![image](https://hackmd.io/_uploads/SyBjKUY4T.png)

In [6]:
#@title 以 Selenium 搜尋 PTT 指定列表

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = Chrome(options=options)
url ='https://www.ptt.cc/bbs/movie/index.html'
wd.get(url)

elem = wd.find_element(By.CLASS_NAME,"title")
print(elem.text)

#beautifulsoup解析
soup = BeautifulSoup(wd.page_source,"html.parser")
links = soup.select('div.title > a')

# title = [ link.get_text() for link in links if '板規' not in link.get_text()]
# href = [ f'https://www.ptt.cc{link.get("href")}' for link in links ]

res = [
    {'title': link.get_text(),
     'href': f'https://www.ptt.cc{link.get("href")}'
     } for link in links if '公告' not in link.get_text() ]


print(res)
wd.close()

[新聞]<X檔案>視后素顏詮釋<鹽路上有你>人生絕境
[{'title': '[新聞]<X檔案>視后素顏詮釋<鹽路上有你>人生絕境', 'href': 'https://www.ptt.cc/bbs/movie/M.1750943011.A.6A9.html'}]


In [7]:
import pandas as pd

pd.DataFrame(res)

,title,href
0,[新聞]<X檔案>視后素顏詮釋<鹽路上有你>人生絕境,https://www.ptt.cc/bbs/movie/M.1750943011.A.6A...


In [8]:
pd.DataFrame(res)['href']

,href
0,https://www.ptt.cc/bbs/movie/M.1750943011.A.6A...




---



In [ ]:
#@title Yahoo搜尋
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from bs4 import BeautifulSoup


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(options=options)
url='https://tw.yahoo.com/'
wd.get(url)

# 操作網頁元素
element = wd.find_element(By.ID, 'header-search-input')
key_word = 'OpenAI' #@param
element.send_keys(key_word)
wd.find_element(By.ID, 'header-desktop-search-button').click()

# 等待目標表格'id 為 web'的div出現
WebDriverWait(wd, 5).until(
    expected_conditions.presence_of_element_located((By.ID, 'web')))

#然後就是beautifulsoup的範疇了，將目前頁面用bs4解析
soup = BeautifulSoup(wd.page_source,"html.parser")
links = soup.select('div#web h3')

for link in links:
    print(link.get_text())

wd.quit()

### ETtoday 新聞雲

In [26]:
# 步驟 1: 安裝與匯入函式庫
# !pip install -q beautifulsoup4 lxml
import requests
from bs4 import BeautifulSoup
import time # 匯入 time 模組以使用 sleep

# 步驟 2: 發送請求，取得網頁 HTML
url = 'https://www.ettoday.net/news/news-list.htm'
headers = { # 模擬真人瀏覽器
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}
response = requests.get(url, headers=headers)

# 確認請求是否成功 (200: 成功, 404: 找不到頁面, 403: 禁止存取)
print(f"請求狀態碼: {response.status_code}")

if response.status_code == 200:
    response.encoding = 'utf-8' # 確保中文正確解碼

    # 步驟 3: 解析 HTML，並擷取目標資料
    soup = BeautifulSoup(response.text, 'lxml')

    # 透過觀察網頁原始碼，我們發現新聞在 <div class="part_list_2"> 裡面的 <h3> 標籤
    news_div = soup.find('div', class_='part_list_2')
    all_h3_tags = news_div.find_all('h3')

    news_data = []
    # 遍歷所有找到的 <h3> 標籤
    for h3 in all_h3_tags:
        title = h3.find('a').text
        # 提取 <a> 標籤的 'href' 屬性值
        link = h3.find('a')['href']
        # ETtoday 的連結是相對路徑，我們需要補上完整的網域名稱
        full_link = f"https://www.ettoday.net{link}"

        news_data.append({'標題': title, '連結': full_link})

    # 步驟 4: 顯示成果
    df_news = pd.DataFrame(news_data)
    print("\n--- ETtoday 即時新聞標題與連結 ---")
    display(df_news.head())
else:
    print("無法成功取得網頁內容。")


請求狀態碼: 200

--- ETtoday 即時新聞標題與連結 ---


,標題,連結
0,李佳穎甜曬一家三口合照 當媽半年吐心聲：越來越有默契,https://www.ettoday.nethttps://star.ettoday.ne...
1,把小吃當正餐超NG！營養師點名「4種人」容易高膽固醇,https://www.ettoday.nethttps://fashion.ettoday...
2,快訊／威力彩17.3億一注獨得 台中今晚誕生新億萬富翁,https://www.ettoday.nethttps://finance.ettoday...
3,林智平11局再見失誤 富邦5比4擊退樂天首度3連勝,https://www.ettoday.nethttps://sports.ettoday....
4,反共是表演 反中才是核心！,https://www.ettoday.nethttps://forum.ettoday.n...


In [27]:
# 步驟 1: 安裝與匯入函式庫
# !pip install -q beautifulsoup4 lxml
import requests
from bs4 import BeautifulSoup
import time # 匯入 time 模組以使用 sleep

# 步驟 2: 發送請求，取得網頁 HTML
url = 'https://www.ettoday.net/news/news-list.htm'
headers = { # 模擬真人瀏覽器
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}
response = requests.get(url, headers=headers)

# 確認請求是否成功 (200: 成功, 404: 找不到頁面, 403: 禁止存取)
print(f"請求狀態碼: {response.status_code}")

if response.status_code == 200:
    response.encoding = 'utf-8' # 確保中文正確解碼

    # 步驟 3: 解析 HTML，並擷取目標資料
    soup = BeautifulSoup(response.text, 'lxml')

    # 透過觀察網頁原始碼，我們發現新聞在 <div class="part_list_2"> 裡面的 <h3> 標籤
    news_div = soup.find('div', class_='part_list_2')
    all_h3_tags = news_div.find_all('h3')

    # 改列表推導式寫法
    news_data = [ {
        '標題': h3.find('a').text,
        '連結': f"https://www.ettoday.net{h3.find('a')['href']}"}
        for h3 in all_h3_tags ]

    # 步驟 4: 顯示成果
    df_news = pd.DataFrame(news_data)
    print("\n--- ETtoday 即時新聞標題與連結 ---")
    display(df_news.head())
else:
    print("無法成功取得網頁內容。")

請求狀態碼: 200

--- ETtoday 即時新聞標題與連結 ---


,標題,連結
0,李佳穎甜曬一家三口合照 當媽半年吐心聲：越來越有默契,https://www.ettoday.nethttps://star.ettoday.ne...
1,把小吃當正餐超NG！營養師點名「4種人」容易高膽固醇,https://www.ettoday.nethttps://fashion.ettoday...
2,快訊／威力彩17.3億一注獨得 台中今晚誕生新億萬富翁,https://www.ettoday.nethttps://finance.ettoday...
3,林智平11局再見失誤 富邦5比4擊退樂天首度3連勝,https://www.ettoday.nethttps://sports.ettoday....
4,反共是表演 反中才是核心！,https://www.ettoday.nethttps://forum.ettoday.n...


In [ ]:
#@title MOMO搶PS5
# https://ithelp.ithome.com.tw/articles/10262268
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


options = Options()
options.add_argument("--headless")
options.add_argument('--no-sandbox') #在背景執行
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--start-maximized")

prefs = {'profile.default_content_setting_values':{'notifications': 2}}
options.add_experimental_option('prefs', prefs)

driver = Chrome(options=options)

driver.get("https://m.momoshop.com.tw/mymomo/login.momo") # 到登入頁面

driver.find_element(By.ID,'memId').send_keys('帳號') # 輸入帳號
driver.find_element(By.ID,'passwd').send_keys('密碼') # 輸入密碼
driver.find_element(By.CLASS_NAME,'login').click()
driver.get("https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9435324")

count=0
while count<2:
  try:
    buy = WebDriverWait(driver, 1, 0.5).until(EC.presence_of_element_located((By.ID, 'buy_yes'))) # 顯性等待
    buy.click() # 偵測到可以購買按鈕就點擊按鈕
    print ('可以購買!')
    break # 後面結帳部分就不寫囉
  except:
    print("還不能購買! 重新整理!")
    driver.refresh() # 重整頁面
    count+=1